# Fetch story samples to analyze

In [3]:
from IPython.display import JSON
import mediacloud.api, datetime, time, json, re, random, os, time, csv
import pandas as pd
import requests
import mcmetadata
import datetime as dt
from dateutil.relativedelta import relativedelta
from importlib.metadata import version
from datetime import date, timedelta
QUERY_LAST_90_DAYS = "publish_date:[NOW-90DAY TO NOW]"
QUERY_LAST_MONTH = "publish_date:[NOW-31DAY TO NOW]"
QUERY_LAST_YEAR = "publish_date:[NOW-1YEAR TO NOW]"
MC_API_KEY = 'INSERT_KEY_HERE'
CID_US_NATIONAL = 34412234
mc_search = mediacloud.api.SearchApi(MC_API_KEY)
mc_search.TIMEOUT_SECS = 5*60
f'Using Media Cloud python client v{version("mediacloud")}'

'Using Media Cloud python client v4.3.0'

In [4]:
from dateutil.rrule import rrule, WEEKLY, TU

# generate dates to study over full year
start_date = dt.date(2024, 1, 1)
end_date = dt.date(2024, 12, 31)

# Generate a list of every Tuesday in the year
selected_dates = list(rrule(freq=WEEKLY, dtstart=start_date, until=end_date, byweekday=TU))

print(f"prepared {len(selected_dates)} dates to study")


prepared 53 dates to study


In [5]:
def get_story_url_file_path(a_date):
    return f"data/story-urls/stories-on-{a_date}.csv"

In [6]:
# fetch list of all stories on single day for each week
for a_date in selected_dates:
    dest_file = get_story_url_file_path(a_date)
    if not os.path.exists(dest_file):
        my_query = '*' 
        day_stories = []
        more_stories = True
        pagination_token = None
        while more_stories:
            page, pagination_token = mc_search.story_list(my_query, a_date.date(), a_date.date(),
                                                        collection_ids=[CID_US_NATIONAL],
                                                        pagination_token=pagination_token)
            day_stories += page
            more_stories = pagination_token is not None
        df = pd.DataFrame(day_stories)
        df.to_csv(dest_file, index=False)
    # load the now-cached data
    df = pd.read_csv(dest_file)
    print(f"{a_date}: {df.shape[0]} stories")

2024-01-02 00:00:00: 6943 stories
2024-01-09 00:00:00: 8861 stories
2024-01-16 00:00:00: 8732 stories
2024-01-23 00:00:00: 9067 stories
2024-01-30 00:00:00: 8914 stories
2024-02-06 00:00:00: 9135 stories
2024-02-13 00:00:00: 9054 stories
2024-02-20 00:00:00: 8374 stories
2024-02-27 00:00:00: 8974 stories
2024-03-05 00:00:00: 9486 stories
2024-03-12 00:00:00: 9118 stories
2024-03-19 00:00:00: 8555 stories
2024-03-26 00:00:00: 8619 stories
2024-04-02 00:00:00: 8426 stories
2024-04-09 00:00:00: 8260 stories
2024-04-16 00:00:00: 8740 stories
2024-04-23 00:00:00: 9390 stories
2024-04-30 00:00:00: 9423 stories
2024-05-07 00:00:00: 10122 stories
2024-05-14 00:00:00: 10411 stories
2024-05-21 00:00:00: 6576 stories
2024-05-28 00:00:00: 8302 stories
2024-06-04 00:00:00: 9489 stories
2024-06-11 00:00:00: 9408 stories
2024-06-18 00:00:00: 9432 stories
2024-06-25 00:00:00: 9308 stories
2024-07-02 00:00:00: 8939 stories
2024-07-09 00:00:00: 8775 stories
2024-07-16 00:00:00: 10091 stories
2024-07-23 

In [13]:
# now pick a 1000 random sample from each set to analyze
sample_dir = "data/sample-story-urls"
os.makedirs(sample_dir, exist_ok=True)

for a_date in selected_dates:
    src_file = get_story_url_file_path(a_date)
    dest_file = f"{sample_dir}/{a_date.date()}.csv"
    if os.path.exists(src_file):
        df = pd.read_csv(src_file)
        sample_df = df.sample(n=min(1000, len(df)), random_state=1)
        sample_df.to_csv(dest_file, index=False)
        print(f"Sampled {len(sample_df)} stories for {a_date}")

Sampled 1000 stories for 2024-01-02 00:00:00
Sampled 1000 stories for 2024-01-09 00:00:00
Sampled 1000 stories for 2024-01-16 00:00:00
Sampled 1000 stories for 2024-01-23 00:00:00
Sampled 1000 stories for 2024-01-30 00:00:00
Sampled 1000 stories for 2024-02-06 00:00:00
Sampled 1000 stories for 2024-02-13 00:00:00
Sampled 1000 stories for 2024-02-20 00:00:00
Sampled 1000 stories for 2024-02-27 00:00:00
Sampled 1000 stories for 2024-03-05 00:00:00
Sampled 1000 stories for 2024-03-12 00:00:00
Sampled 1000 stories for 2024-03-19 00:00:00
Sampled 1000 stories for 2024-03-26 00:00:00
Sampled 1000 stories for 2024-04-02 00:00:00
Sampled 1000 stories for 2024-04-09 00:00:00
Sampled 1000 stories for 2024-04-16 00:00:00
Sampled 1000 stories for 2024-04-23 00:00:00
Sampled 1000 stories for 2024-04-30 00:00:00
Sampled 1000 stories for 2024-05-07 00:00:00
Sampled 1000 stories for 2024-05-14 00:00:00
Sampled 1000 stories for 2024-05-21 00:00:00
Sampled 1000 stories for 2024-05-28 00:00:00
Sampled 10